### **Graph 1**

#### Google Drive 연결

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


#### 사용할 module import

In [92]:
import pandas as pd
import networkx as nx
from sklearn.metrics.pairwise import cosine_similarity

#### 데이터 불러오기
**# Columns Information**
- painting: 작품명
- emotion: 감정
- utterance: 작품 설명
- art_style: 작품 스타일
- repetition: 언급 수
- artist: 작가
- year: 출품 년도
- Vectorization: 벡터화

In [112]:
# CSV 파일 불러오기
nodes = pd.read_csv('/content/drive/MyDrive/2024-1 graph-mining/final_project/Contrastive_with_vectors_280.csv')

#### [Node] : painting
- 미술 작품 2500개

In [113]:
# 노드 데이터프레임에서 노드 리스트 생성
nodes_list = nodes['painting'].tolist()

#### [Edge] : emotion
- 감정적 유사성을 가진 작품 → 같은 감정을 표현하고 있는 작품들끼리 연결

In [114]:
# 엣지 생성을 위한 데이터프레임 생성
edges_df = nodes[['painting', 'emotion']].copy()

#### 그래프 생성
- Node와 Edge 연결
- Node의 attribute 추가 -> art_style 별로 노드 색상 부여

In [115]:
# 노드 리스트를 딕셔너리 형태로 변환하여 노드 ID로 사용
nodes_dict = {node: idx for idx, node in enumerate(nodes_list)}

In [116]:
# 엣지를 연결할 그래프 객체 생성
G = nx.Graph()

In [117]:
# 노드 추가 및 속성 설정
for idx, row in nodes.iterrows():
    painting = row['painting']
    art_style = row['art_style']
    G.add_node(painting, art_style=art_style)

In [118]:
# 엣지 추가
for _, row in edges_df.iterrows():
    painting = row['painting']
    emotion = row['emotion']
    # 같은 emotion을 가진 노드를 연결
    connected_nodes = edges_df[edges_df['emotion'] == emotion]['painting'].tolist()
    for connected_node in connected_nodes:
        if painting != connected_node:  # 자기 자신과는 연결하지 않도록 조건 추가
            G.add_edge(painting, connected_node)

#### Edge Weight 추가
- Vectorization column의 값을 활용하여 노드 간 Cosine Similarity 계산


In [119]:
# 각 노드의 벡터화된 값을 가져오는 함수 정의
def get_vector(node_id):
    vector_str = nodes.loc[nodes['painting'] == node_id, 'Vectorization'].values[0]
    # 문자열을 부동 소수점 배열로 변환
    vector = np.array([float(x) for x in vector_str.strip('[]').split()])
    return vector

# 각 노드 간의 코사인 유사도 계산하여 가중치로 추가하는 함수 정의
def add_edge_weights(graph):
    for u, v in graph.edges():
        vector_u = get_vector(u)
        vector_v = get_vector(v)
        similarity = cosine_similarity([vector_u], [vector_v])[0][0]
        graph[u][v]['weight'] = similarity

# GraphML 파일로 저장하는 함수 정의
def save_graphml(graph, filename):
    nx.write_graphml_lxml(graph, filename)

In [120]:
# 가중치가 부여된 그래프 생성
add_edge_weights(G)

#### 생성된 Node, Edge 개수 확인

In [121]:
# 노드 개수 출력
print("Number of nodes:", G.number_of_nodes())

# 엣지 개수 출력
print("Number of edges:", G.number_of_edges())

Number of nodes: 272
Number of edges: 9807


#### GraphML 파일로 저장

In [122]:
# GraphML 파일로 저장
save_graphml(G, "artwork_graph_with_weights.graphml")